# Dog of Wolf - 推論

In [1]:
from glob import glob

import pytorch_lightning as pl
import torch
import torch.nn as nn
from natsort import natsorted
from PIL import Image
from torchvision import transforms
from torchvision.models import resnet18


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## モデル

In [3]:
class Net(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.feature = resnet18()
        self.fc = nn.Linear(1000, 2)

    def forward(self, x):
        h = self.feature(x)
        h = self.fc(h)
        return h

In [4]:
# モデルを評価モードに設定
net = Net().cpu().eval()

# 保存されたモデルをCPU上でロード
net.load_state_dict(torch.load('./model_resnet18_state.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

## 予測

In [5]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
paths = natsorted(glob('../data/test/dog/*.png') + glob('../data/test/wolf/*.png'))
imgs = [transform(Image.open(path).convert('RGB')) for path in paths]
x_test = torch.stack(imgs)
paths

['../data/test/dog/dog-21.png',
 '../data/test/dog/dog-22.png',
 '../data/test/dog/dog-23.png',
 '../data/test/dog/dog-24.png',
 '../data/test/dog/dog-25.png',
 '../data/test/wolf/wolf-21.png',
 '../data/test/wolf/wolf-22.png',
 '../data/test/wolf/wolf-23.png',
 '../data/test/wolf/wolf-24.png',
 '../data/test/wolf/wolf-25.png']

In [9]:
# 勾配計算を無効化
with torch.no_grad():
  y = net(x_test)

clazz = torch.argmax(y, dim=1)
clazz

tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])